<a href="https://colab.research.google.com/github/amirhossini/Pyomo-Educational-Notebooks/blob/main/Pyomo4_MINLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyomo Examples
__Notebook:__ Mixed Integer Nonlinear Programming (MINLP)

__Questions:__ amir.hossini@queensu.ca

_Libraies_

In [2]:
!pip install pyomo
import pyomo.environ as pyomo

import numpy as np
import pandas as pd

import os

     |████████████████████████████████| 9.6 MB 14.3 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 


# Example 1 - Fuel scheduling and unit commitment

In [4]:
model = pyomo.ConcreteModel();

# Set definition
model.t = pyomo.RangeSet(1,3);

# Variable declaration
model.b = pyomo.Var(model.t,domain = pyomo.Binary,doc = 'Status of operation of oil-based generation unit in each period');
model.V = pyomo.Var(model.t,domain = pyomo.NonNegativeReals,bounds=(0,4000),doc = 'Volume of oil in storage tank in each period');
model.x = pyomo.Var(model.t,domain = pyomo.NonNegativeReals,doc = 'Oil consumption in each period');
model.y = pyomo.Var(model.t,domain = pyomo.NonNegativeReals,bounds=(50,700),doc = 'Power generated by other units in each period');
model.h = pyomo.Var(model.t,domain = pyomo.NonNegativeReals,doc = 'Power generated by oil-based generation unit in each period');

# Parameter declaration
model.lam = pyomo.Param(model.t,initialize = {1:400,2:900,3:700});
model.mu = pyomo.Param(initialize = 3000);

# Constraint declaration
def rule1(model,t):
  return model.h[t] >= 100*model.b[t]
model.eq1 = pyomo.Constraint(model.t,rule=rule1,doc = 'Minimum level of power generation by oil-based generation unit in each period');

def rule2(model,t):
  return model.h[t] <= 500*model.b[t]
model.eq2 = pyomo.Constraint(model.t,rule=rule2,doc = 'Maximum level of power generation by oil-based generation unit in each period');

def rule3(model,t):
  if t==1:
    return model.V[t] == 500 - model.x[t] + model.mu
  else:
    return model.V[t] == model.V[t-1] + 500 - model.x[t]
model.eq3 = pyomo.Constraint(model.t,rule = rule3,doc = 'Volume balance for oil storage tank in each period');

def rule4(model,t):
  return model.x[t] == 50*model.b[t] + model.h[t] + 0.005*np.power(model.h[t],2)
model.eq4 = pyomo.Constraint(model.t,rule=rule4,doc='Calculation of power generation from oil consumption in each period');

def rule5(model,t):
  return model.h[t] + model.y[t] >= model.lam[t]
model.eq5 = pyomo.Constraint(model.t,rule=rule5,doc = 'Total power generated by oil-based, as well as other units, must meet demand in esch period');

def rule6(model,t):
  return model.V[3] >= 2000
model.eq6 = pyomo.Constraint(model.t,rule=rule6,doc='Minimum volume of oil in storage tank in period 3');

# Objective function
def obj_rule(model):
  return sum(300 + 6*model.y[t] + 0.0025*np.power(model.y[t],2) for t in model.t)
model.obj = pyomo.Objective(rule=obj_rule,sense=pyomo.minimize);

# PLEASE REPLACE THE **** WITH YOUR EMAIL ID: for example, youremailid@something.com
os.environ['NEOS_EMAIL'] = 'amir.hossini@queensu.ca';
solver_manager = pyomo.SolverManagerFactory('neos');

results = solver_manager.solve(model,opt='minlp');

results.write()
print("\nRESULTS:");
print("\nTotal cost = ",model.obj());

for t in model.t:
  print("\nStatus of operation of oil-based generation unit in period",t,"=",model.b[t]());

for t in model.t:
  print("\nVolume of oil in storage tank in period",t,"=",model.V[t](),"bbl");

for t in model.t:
  print("\nOil consumption in period",t,"=",model.x[t](),"bbl");

for t in model.t:
  print("\nPower generated by other units in period",t,"=",model.y[t](),"MW");

for t in model.t:
  print("\nPower generated by oil-based generation unit in period",t,"=",model.h[t](),"MW");
  

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 15
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: MINLP-B&B (20100607)\x3a Optimal solution found; 7 subproblems, objective = 8566.118961687665; Evals\x3a obj = 31, constr = 47, grad = 47, Hes = 49
  Termination condition: optimal
  Id: 0
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 


# Example 2 - Agricultural Pricing

In [ ]:
model = pyomo.ConcreteModel();

# Set
model.p = pyomo.Set(initialize = ['milk','butter','cheese1','cheese2']);

# Variable declaration
model.x = pyomo.Var(model.p,domain = pyomo.NonNegativeIntegers);
model.c = pyomo.Var(model.p,domain = pyomo.NonNegativeReals);

# Parameter declaration
data = pyomo.DataPortal();
data.load(filename='Pyomo_MINLP_example_2_input_data.json');

model.alpha = pyomo.Param(model.p,initialize = data['alpha'],doc = 'Fat content');
model.beta = pyomo.Param(model.p,initialize = data['beta'],doc = 'Dry matter content')

model.mu = pyomo.Param(model.p,initialize = data['mu'],doc = 'Average demand');
model.chi = pyomo.Param(model.p,initialize = data['chi'],doc = 'Average pricing');

model.epsilon = pyomo.Param(model.p,initialize = data['epsilon'],doc = 'Elasticity');

model.rho = pyomo.Param(model.p,initialize = data['rho'],doc = 'Coefficients for policy constraint');

model.nu = pyomo.Param(model.p,model.p,initialize = data['nu'],doc = 'Cross elasticity');

model.gamma = pyomo.Param(initialize = 121);
model.theta = pyomo.Param(initialize = 250);

# Constraint declaration
def rule1(model,p):
  return sum(model.alpha[p]*model.x[p]/100 for p in model.p) <= model.gamma
model.eq1 = pyomo.Constraint(model.p,rule=rule1,doc = 'Fat content constraint');

def rule2(model,p):
  return sum(model.beta[p]*model.x[p]/100 for p in model.p) <= model.theta
model.eq2 = pyomo.Constraint(model.p,rule=rule2,doc = 'Dry matter content');

def rule3(model,p):
  return (model.x['milk']-model.mu['milk'])/model.mu['milk'] == -model.epsilon['milk']*(model.c['milk']-model.chi['milk'])/model.chi['milk']
model.eq3 = pyomo.Constraint(model.p,rule=rule3,doc = 'Linear demand relationship for milk');

def rule4(model,p):
  return (model.x['butter']-model.mu['butter'])/model.mu['butter'] == -model.epsilon['butter']*(model.c['butter']-model.chi['butter'])/model.chi['butter']
model.eq4 = pyomo.Constraint(model.p,rule=rule4,doc = 'Linear demand relationship for butter');

def rule5(model,p):
  return (model.x['cheese1']-model.mu['cheese1'])/model.mu['cheese1'] == -model.epsilon['cheese1']*(model.c['cheese1']-model.chi['cheese1'])/model.chi['cheese1'] + model.nu['cheese1','cheese2']*(model.c['cheese2']-model.chi['cheese2'])/model.chi['cheese2']
model.eq5 = pyomo.Constraint(model.p,rule=rule5,doc = 'Linear demand relationship for cheese1');

def rule6(model,p):
  return (model.x['cheese2']-model.mu['cheese2'])/model.mu['cheese2'] == -model.epsilon['cheese2']*(model.c['cheese2']-model.chi['cheese2'])/model.chi['cheese2'] + model.nu['cheese2','cheese1']*(model.c['cheese1']-model.chi['cheese1'])/model.chi['cheese1']
model.eq6 = pyomo.Constraint(model.p,rule=rule6,doc = 'Linear demand relationship for cheese2');

def rule7(model,p):
  return sum(model.rho[p]*(model.c[p]-model.chi[p])/model.chi[p] for p in model.p) == 0
model.eq7 = pyomo.Constraint(model.p,rule=rule7,doc = 'Policy constraint');

#Objective function declaration
def obj_rule(model):
  return sum(model.x[p]*model.c[p] for p in model.p)
model.obj = pyomo.Objective(rule=obj_rule,sense=pyomo.maximize);

# PLEASE REPLACE THE **** WITH YOUR EMAIL ID: for example, youremailid@something.com
os.environ['NEOS_EMAIL'] = '****';
solver_manager = pyomo.SolverManagerFactory('neos');

results = solver_manager.solve(model,opt='couenne');

results.write();
print("\nRESULTS:");
print("\nTotal revenue: ",model.obj());

for p in model.p:
  print("\nAmount of product (",p,") consumed =",model.x[p](),"ktons");

for p in model.p:
  print("\nPrice of product (",p,") =",model.c[p](),"(in 1000s of pounds)");
  

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: couenne\x3a Infeasible problem
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 8
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Message: couenne\x3a Infeasible problem
  Termination condition: infeasible
  Id: 220
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------------------

# Example 3 - Yield Management

In [ ]:
model = pyomo.ConcreteModel();

# Set definition
model.c = pyomo.Set(initialize = ['First','Business','Economy']);
model.h = pyomo.Set(initialize = ['Option1','Option2','Option3']);
model.i = pyomo.Set(initialize = ['Scenario1','Scenario2','Scenario3']);

# Creating alias sets for model.i
model.ii = pyomo.Set(initialize = model.i);
model.iii = pyomo.Set(initialize = model.i);

# Variable declaration
model.p1 = pyomo.Var(model.c,model.h,domain = pyomo.Binary,doc = 'Whether price option h is chosen for ticket class c in week 1');
model.p2 = pyomo.Var(model.i,model.c,model.h,domain = pyomo.Binary,doc = 'Whether price option h is chosen for ticket class c in week 2, with demand scenario i occuring in week 1');
model.p3 = pyomo.Var(model.i,model.ii,model.c,model.h,domain = pyomo.Binary,doc = 'Whether price option h is chosen for ticket class c in week 3, with demand scenario i and ii occuring in week 1 and 2, respectively');

model.s1 = pyomo.Var(model.i,model.c,model.h,domain = pyomo.NonNegativeReals,doc = 'Number of tickets sold of type c with price option h in week 1 with demand scenario i');
model.s2 = pyomo.Var(model.i,model.ii,model.c,model.h,domain = pyomo.NonNegativeReals,doc = 'Number of tickets sold of type c with price option h in week 2 with demand scenario i in week 1 and ii in week 2');
model.s3 = pyomo.Var(model.i,model.ii,model.iii,model.c,model.h,domain = pyomo.NonNegativeReals,doc = 'Number of tickets sold of type c with price option h in week 3 with demand scenarios i, ii, iii in weeks 1, 2 and 3, respectively');

model.n = pyomo.Var(domain = pyomo.NonNegativeIntegers,bounds = (0,6));

# Parameter declaration
model.psi = pyomo.Param(model.i,initialize = {'Scenario1':0.1,'Scenario2':0.7,'Scenario3':0.2});

delta1 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0,1],usecols='A:E',nrows=9).fillna('');

delta2 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0,1],usecols='G:K',nrows=9).fillna('');
delta2 = delta2.rename(columns={"Option1.1":"Option1","Option2.1":"Option2","Option3.1":"Option3"});

delta3 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0,1],usecols='M:Q',nrows=9).fillna('');
delta3 = delta3.rename(columns={"Option1.2":"Option1","Option2.2":"Option2","Option3.2":"Option3"});

pi1 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0],usecols='A:D',skiprows=12).fillna('');

pi2 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0],usecols='G:J',skiprows=12).fillna('');
pi2 = pi2.rename(columns={"Option1.1":"Option1","Option2.1":"Option2","Option3.1":"Option3"});

pi3 = pd.read_excel('Pyomo_MINLP_example_3_input_data.xls',sheet_name='Sheet1',header=1,index_col=[0],usecols='M:P',skiprows=12).fillna('');
pi3 = pi3.rename(columns={"Option1.2":"Option1","Option2.2":"Option2","Option3.2":"Option3"});

model.gamma = pyomo.Param(model.c,initialize = {'First':37,'Business':38,'Economy':47});

# Constraint declaration
def rule1a(model,c):
  return sum(model.p1[c,h] for h in model.h) == 1
model.eq1a = pyomo.Constraint(model.c,rule=rule1a,doc = 'Only one price option can be chosen in week 1');

def rule1b(model,i,c,h):
  return model.s1[i,c,h] <= delta1[h][i][c]*model.p1[c,h]
model.eq1b = pyomo.Constraint(model.i,model.c,model.h,rule=rule1b,doc = 'Sales must not exceed demand in week 1');

def rule2a(model,i,c):
  return sum(model.p2[i,c,h] for h in model.h) == 1
model.eq2a = pyomo.Constraint(model.i,model.c,rule=rule2a,doc = 'Only one price option can be chosen in week 2');

def rule2b(model,i,ii,c,h):
  return model.s2[i,ii,c,h] <= delta2[h][ii][c]*model.p2[i,c,h]
model.eq2b = pyomo.Constraint(model.i,model.ii,model.c,model.h,rule=rule2b,doc = 'Sales must not exceed demand in week 2');

def rule3a(model,i,ii,c):
  return sum(model.p3[i,ii,c,h] for h in model.h) == 1
model.eq3a = pyomo.Constraint(model.i,model.ii,model.c,rule=rule3a,doc = 'Only one price option can be chosen in week 3');

def rule3b(model,i,ii,iii,c,h):
  return model.s3[i,ii,iii,c,h] <= delta3[h][iii][c]*model.p3[i,ii,c,h]
model.eq3b = pyomo.Constraint(model.i,model.ii,model.iii,model.c,model.h,rule=rule3b,doc = 'Sales must not exceed demand in week 3');

def rule4(model,i,ii,iii,c):
  return sum(model.s1[i,c,h] for h in model.h) + sum(model.s2[i,ii,c,h] for h in model.h) + sum(model.s3[i,ii,iii,c,h] for h in model.h) <= model.gamma[c]*model.n
model.eq4 = pyomo.Constraint(model.i,model.ii,model.iii,model.c,rule=rule4,doc = 'Seat capacity must be abided by');

# Objective function definition
def obj_rule(model):
  term1 = sum(sum(sum(model.psi[i]*pi1[h][c]*model.p1[c,h]*model.s1[i,c,h] for i in model.i) for c in model.c) for h in model.h)
  term2 = sum(sum(sum(sum(model.psi[i]*model.psi[ii]*pi2[h][c]*model.p2[i,c,h]*model.s2[i,ii,c,h] for i in model.i) for ii in model.ii) for c in model.c) for h in model.h)
  term3 = sum(sum(sum(sum(sum(model.psi[i]*model.psi[ii]*model.psi[iii]*pi3[h][c]*model.p3[i,ii,c,h]*model.s3[i,ii,iii,c,h] for i in model.i) for ii in model.ii) for iii in model.iii) for c in model.c) for h in model.h)
  term4 = 50000*model.n
  return term1 + term2 + term3 - term4
model.obj = pyomo.Objective(rule=obj_rule,sense=pyomo.maximize);

# PLEASE REPLACE THE **** WITH YOUR EMAIL ID: for example, youremailid@something.com
os.environ['NEOS_EMAIL'] = '****';
solver_manager = pyomo.SolverManagerFactory('neos');

results = solver_manager.solve(model,opt='bonmin');

results.write()

print("\nRESULTS:");
print("\nTotal revenue: ",model.obj());
print("\nNumber of planes to book = ",model.n());

print("\nWeek 1 provisional prices: ");
for c in model.c:
  for h in model.h:
    if model.p1[c,h].value != 0:
      print(c,"class tickets with Price",h,"=",pi1[h][c],"pounds");

print("\nWeek 2 provisional prices: ");
for i in model.i:
  for c in model.c:
    for h in model.h:
      if model.p2[i,c,h].value != 0:
        print(i,"=>",c,"class tickets with Price",h,"=",pi2[h][c],"pounds");

print("\nWeek 3 provisional prices: ");
for i in model.i:
  for ii in model.ii:
    for c in model.c:
      for h in model.h:
        if model.p3[i,ii,c,h].value != 0:
          print("(Week1,",i,")","(Week2,",ii,") => ",c,"class tickets with Price",h," = ",pi3[h][c],"pounds");

print("\nNumber of tickets booked in week 1: ");
for i in model.i:
  for c in model.c:
    for h in model.h:
      print("(Week1",i,"),",c,h," => ",model.s1[i,c,h].value);

print("\nNumber of tickets booked in week 2: ");
for i in model.i:
  for c in model.ii:
    for c in model.c:
      for h in model.h:
        print("(Week1,",i,"), (Week2,",ii,"),",c,h," => ",model.s2[i,ii,c,h].value);

print("\nNumber of tickets booked in week 3: ");
for i in model.i:
  for ii in model.ii:
    for iii in model.iii:
      for c in model.c:
        for h in model.h:
          print("(Week1,",i,"), (Week2,",ii,"), (Week3,",iii,"),",c,h," => ",model.s3[i,ii,iii,c,h].value);


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 469
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

RESULTS:

Total revenue:  130695.30011217634

Number of planes to bo